In [1]:
import pandas as pd
import numpy as np

In [6]:
# Load the Activity Metrics CSV file
activity_metrics_df = pd.read_csv("AMKEY_GoldenStandard.csv")
activity_metrics_df.head(20)

,AMKEY,ActivityMetric
0,3,Advisory fees as per income statement
1,6,Air emissions of the following pollutants: (1) CO
2,7,Air emissions of the following pollutants: (2)...
3,8,Air emissions of the following pollutants: (3)...
4,9,Air emissions of the following pollutants: (4)...
5,10,Air emissions of the following pollutants: (5)...
6,11,ALL Administration expenses per income statement
7,12,All Inury Frequency Rate (Injuries/1m hrs worked)
8,13,"Amount of assets under management, by asset cl..."
9,14,"Amount of assets under management, by asset cl..."


In [7]:
# Load the second set of Activity Metrics CSV file
activity_metrics_set2 = pd.read_csv("ActivityMetricsSynonyms.csv")
activity_metrics_set2.head(20)

,AMKEY,Group,ActivityMetric,ClientMetric
0,7,Impala,Air emissions of the following pollutants: (2)...,Total indirect Nox
1,7,Sasol,Air emissions of the following pollutants: (2)...,Nitrogen oxides (NOx) (kilotons)
2,8,Sasol,Air emissions of the following pollutants: (3)...,Sulphur oxides (SOx ) (kilotons)
3,8,Ssw,Air emissions of the following pollutants: (3)...,SO2 emissions
4,8,Impala,Air emissions of the following pollutants: (3)...,Total direct SO2 + Total indirect SO2
5,9,Sasol,Air emissions of the following pollutants: (4)...,Particulates (fly ash) (kilotons)
6,10,Sasol,Air emissions of the following pollutants: (5)...,Volatile Organic Compounds (VOC) Indicator of ...
7,12,Tongaat,All Inury Frequency Rate (Injuries/1m hrs worked),Total injury frequency rate (TIFR) – employees...
8,27,Ssw,Area of forestland managed by the entity,Land under management
9,27,Impala,Area of forestland managed by the entity,Leased and managed land - South Africa


In [2]:
from pdfminer.high_level import extract_text
import os

# Function to extract text from a PDF file using pdfminer
def extract_text_from_pdf(pdf_file):
    text = extract_text(pdf_file)
    return text

# List of PDFs with corresponding IDs and file names
pdf_files = [
    {'ID': 'Oceana1', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Multiple source data\Oceana_Group_Sustainability_Report_2022.pdf'},
    {'ID': 'Ssw', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Unstructured data\ssw-IR22.pdf'},
    {'ID': 'Sasol', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\SASOL Sustainability Report 2023 20-09_0.pdf'},
    {'ID': 'Impala', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\ESG-spreads.pdf'},
    {'ID': 'Clicks', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\Clicks-Sustainability-Report-2022.pdf'},
    {'ID': 'Oceana2', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Multiple source data\Oceana_ESG_Databook_FY2022.pdf'},
    {'ID': 'Uct2', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Multiple source data\afs2021.pdf'},
    {'ID': 'Picknpay', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\picknpay-esg-report-spreads-2023.pdf'},
    {'ID': 'Distell', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\DISTELL ESG Appendix 2022.pdf'},
    {'ID': 'Uct1', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Multiple source data\UCT_Carbon_Footprint_Report_2020-2021.pdf'},
    {'ID': 'Absa', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\2022-Absa-Group-limited-Environmental-Social-and-Governance-Data-sheet.pdf'},
    {'ID': 'Tongaat', 'File Name': r'c:\Users\hp\Victor-Files\unify_documents\Structured data\2021ESG.pdf'}
]

# Directory to store extracted text files
output_directory = 'extracted_text_pdfminer'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Loop through each PDF file, extract text, and save to a text file
for pdf_info in pdf_files:
    pdf_id = pdf_info['ID']
    pdf_file_name = pdf_info['File Name']
    
    # Extract text
    extracted_text = extract_text_from_pdf(pdf_file_name)
    
    # Save extracted text to a text file
    output_file_path = os.path.join(output_directory, f'{pdf_id}_extracted_text_pdfminer.txt')
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(extracted_text)

    print(f'Text extracted from {pdf_id}: {output_file_path}')


Text extracted from Oceana1: extracted_text_pdfminer\Oceana1_extracted_text_pdfminer.txt
Text extracted from Ssw: extracted_text_pdfminer\Ssw_extracted_text_pdfminer.txt
Text extracted from Sasol: extracted_text_pdfminer\Sasol_extracted_text_pdfminer.txt
Text extracted from Impala: extracted_text_pdfminer\Impala_extracted_text_pdfminer.txt
Text extracted from Clicks: extracted_text_pdfminer\Clicks_extracted_text_pdfminer.txt
Text extracted from Oceana2: extracted_text_pdfminer\Oceana2_extracted_text_pdfminer.txt
Text extracted from Uct2: extracted_text_pdfminer\Uct2_extracted_text_pdfminer.txt
Text extracted from Picknpay: extracted_text_pdfminer\Picknpay_extracted_text_pdfminer.txt
Text extracted from Distell: extracted_text_pdfminer\Distell_extracted_text_pdfminer.txt
Text extracted from Uct1: extracted_text_pdfminer\Uct1_extracted_text_pdfminer.txt
Text extracted from Absa: extracted_text_pdfminer\Absa_extracted_text_pdfminer.txt


FileNotFoundError: [Errno 2] No such file or directory: 'extracted_text_pdfminer\\Tongaat_extracted_text_pdfminer.txt'

In [ ]:
extract_text

<function pdfminer.high_level.extract_text(pdf_file, password='', page_numbers=None, maxpages=0, caching=True, codec='utf-8', laparams=None)>

In [ ]:
import pdfplumber
import pandas as pd


# List of annual reports with corresponding file names
annual_reports = [
    ("Oceana1.pdf", "Oceana_Group_Sustainability_Report_2022.pdf"),
    ("Ssw.pdf", "ssw-IR22.pdf"),
    ("Sasol.pdf", "SASOL Sustainability Report 2023 20-09_0.pdf"),
    ("Impala.pdf", "ESG-spreads.pdf"),
    ("Clicks.pdf", "Clicks-Sustainability-Report-2022.pdf"),
    ("Oceana2.pdf", "Oceana_ESG_Databook_FY2022.pdf"),
    ("Uct2.pdf", "afs2021.pdf"),
    ("Picknpay.pdf", "picknpay-esg-report-spreads-2023.pdf"),
    ("Distell.pdf", "DISTELL ESG Appendix 2022.pdf"),
    ("Uct1.pdf", "UCT_Carbon_Footprint_Report_2020-2021.pdf"),
    ("Absa.pdf", "2022-Absa-Group-limited-Environmental-Social-and-Governance-Data-sheet.pdf"),
    ("Tongaat.pdf", "2021ESG.pdf")
]

# Loop through the annual reports and extract text relevant to Activity Metrics
for report_id, file_name in annual_reports:
    with pdfplumber.open(file_name) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()

        # Extract information only for relevant Activity Metrics
        for _, row in activity_metrics_df.iterrows():
            amkey = row["AMKEY"]
            group = row["Group"]
            activity_metric = row["ActivityMetric"]
            client_metric = row["ClientMetric"]

            # Check if the activity metric information is present in the report
            if amkey in text and group in text and activity_metric in text and client_metric in text:
                # Extract and process the relevant information as needed
                relevant_info = text[text.find(activity_metric):text.find(client_metric)]
                print(f"Document ID: {report_id}, AMKEY: {amkey}")
                print(relevant_info)
                print("=" * 50)
